In [29]:
import tkinter as tk
from pytube import YouTube 
from bs4 import BeautifulSoup
import requests
import threading

import subprocess as sp

In [ ]:
def get_urls(url):
    urls = []
    if '&list=' not in url : return urls
    response = requests.get(url)
    if response.status_code != 200:
        print('請求失敗')
        return
    
    bs = BeautifulSoup(response.txt, "lxml")
    a_list = bs.find_all('a')
    base = 'https://www.youtube.com/'
    for a in a_list:
        href = a.get('href')
        url = base + href
        if('&index=' in url) and (url not in urls):
            urls.append(url)
    return urls

In [26]:
lock = threading.Lock()
def start_dload(url, listbox):
    try:
        yt = YouTube(url)
    except:
        print("pytube 不支援此影片或網址錯誤")
        return
    name = yt.title
    
    lock.acquire()
    no = listbox.size()
    listbox.insert(tk.END, f'{no:02d}:{name}.....下載中')
    print('插入:' , no, name)
    lock.release()
    
    yt.stream.first().download()
    
    lock.acquire()
    print('更新:', no, name)
    listbox.delete(no)
    listbox.insert(no, f'{no:02d}:{name}.....下載完成' + ret)
    print(f'{name}......下載完成')
    lock.release()
    


In [27]:
window = tk.Tk()
window.geometry('640x480')
window.title('Youtube 急速下載器')

input_fm = tk.Frame(window, bg='red', width=640, height=120)

input_fm.pack()

lb = tk.Label(input_fm, bg='red' , fg='white', text='請輸入Youtube網址', font=('新細體', 30))
lb.place(rely=0.25, relx=0.5, anchor='center')

yt_url = tk.StringVar()
entry = tk.Entry(input_fm, textvariable=yt_url, width=50)
entry.place(rely=0.5, relx=0.5, anchor='center')

def click_func():
    url = yt_url.get()
    try:
        YouTube(url)
    except:
        messagebox.showerror('錯誤', 'pytube不支援此影片')
        return
    urls = get_urls(url)
    
    if urls and messagebox.askyesno('確認方塊', '是否下載影片中的所有影片'):
        print('開始下載清單')
        
        for u in urls:
            threading.thread(target=start_dload, args=(u, listbox)).start()
    else:
        yt = YouTube(url)
        
        if messagebox.askyesno('詢問方塊', f'是否下載 {yt.title} 影片'):
            threading.thread(target=start_dload, args=(url, listbox)).start()
        else:
            print('取消下載')

        
    

btn = tk.Button(input_fm, text="下載影片", command=click_func, bg="#FFD700", fg="black", font=("細明體", 10))
btn.place(rely=0.5, relx=0.85, anchor='center')

dload_fm = tk.Frame(window, width=640, height=480-120)
dload_fm.pack()

lb = tk.Label(dload_fm, text='下載狀態', fg='black', font=('新細體', 10))
lb.place(rely=0.1, relx=0.5, anchor='center')

list_box = tk.Listbox(dload_fm, width=65, height=15)
list_box.place(rely=0.5, relx=0.5, anchor='center')

sbar = tk.Scrollbar(dload_fm)
sbar.place(rely=0.5, relx=0.87, anchor="center", relheight=0.7)

list_box.config(yscrollcommand = sbar.set)
sbar.config(command=list_box.yview)
window.mainloop()

In [30]:
def yget_info(url):
    process = sp.Popen('you-get -i' + url, shell=True, stdout=sp.PIPE, stderr=sp.PIPE)
    r = process.communicate()
    s = str(r[0], 'utf-8')
    print(s)
    if s.find('title:') < 0:
        return '', ''
    
    title = s[s.find("title:")+6 : s.find("streams")].strip()
    itag = s[s.find("itage:")+6 : s.find("container")].strip()
    
    if len(itage) > 8:
        itag = itag[4 : -4]
        
    return title, itag
    

In [ ]:
def yget_dl(url itag=None):
    cmd = 'you-get'
    if itag:
        cmd = cmd + '--itag' + itag + ' '
    process = sp.Popen(cmd+url)
    process.wait()
    return process.returncode